In [1]:
import numpy as np
import cv2
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [28]:
background = None
accumulated_weight = 0.5

#ROI frame co-ordinates...
ROI_top = 70
ROI_bottom = 370
ROI_right = 250
ROI_left = 550

In [29]:
def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [30]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY_INV)
    
    #Fetching contours in the frame (These contours can be of hand or any other object in foreground)
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # If length of contours list = 0, means we didn't get any contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand 
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and the
        # thresholded image of hand...
        return (thresholded, hand_segment_max_cont)

In [40]:
cam = cv2.VideoCapture(0)
num_frames =0

if not os.path.exists("data"):
    os.makedirs("data")
    os.makedirs("data/test")
    os.makedirs("data/train/Scissor")
    os.makedirs("data/train/Wait")
    os.makedirs("data/train/Rock")
    os.makedirs("data/train/Paper")

# Data collection for train or test or validation...
mode = 'train'
directory = 'data/'+mode+'/'


while True:
    ret, frame = cam.read()
    
    # flipping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)
    
    #Count the numbers of images in the respective directory...
    count = {'Scissor': len(os.listdir(directory+"/Scissor")),
             'Rock': len(os.listdir(directory+"/Rock")),
             'Paper': len(os.listdir(directory+"/Paper")),
             'Wait': len(os.listdir(directory+"/Wait"))
            }
    
    # Printing the count in each set to the screen...
    cv2.putText(frame, "MODE : "+mode, (10, 100), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    cv2.putText(frame, "IMAGE COUNT", (10, 150), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    cv2.putText(frame, "Rock : "+str(count['Rock']), (10, 170), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    cv2.putText(frame, "Paper : "+str(count['Paper']), (10, 190), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    cv2.putText(frame, "Scissor : "+str(count['Scissor']), (10, 210), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    cv2.putText(frame, "Wait : "+str(count['Wait']), (10, 230), cv2.FONT_HERSHEY_PLAIN, 1.5, (238,229,172), 2)
    
    frame_copy = frame.copy()
    
    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    if num_frames < 70:
        
        #Fetching the background...
        cal_accum_avg(gray_frame, accumulated_weight)
        cv2.putText(frame_copy, "Please wait...Detecting Background...", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            
            #Display the thresholded Hand image...
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (224, 224))
            
    # Draw rectangular frame on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (240,140,70), 3)
    
    # Incrementing the number of frames for tracking
    num_frames += 1
    
    # Display the frame with segmented hand
    cv2.putText(frame_copy, "Hand sign detecting...", (180, 40), cv2.FONT_ITALIC, 1.0, (186,70,0), 2)
    cv2.imshow("Sign detection", frame_copy)
    
    # Close windows with Esc
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == 27: # esc key
        break
    if interrupt & 0xFF == ord('r'):
        cv2.imwrite(directory+'Rock/'+str(count['Rock'])+'.jpg', thresholded)
    if interrupt & 0xFF == ord('p'):
        cv2.imwrite(directory+'Paper/'+str(count['Paper'])+'.jpg', thresholded)
    if interrupt & 0xFF == ord('s'):
        cv2.imwrite(directory+'Scissor/'+str(count['Scissor'])+'.jpg', thresholded)
    if interrupt & 0xFF == ord('w'):
        cv2.imwrite(directory+'Wait/'+str(count['Wait'])+'.jpg', thresholded)
    
# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()